In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/My Drive

/content/drive/My Drive


In [ ]:
! pip install albumentations==0.4.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 117 kB 5.0 MB/s 
     |████████████████████████████████| 948 kB 38.2 MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65174 sha256=ff131663ad6d72e97c9561b3ef90fc0b149391ceff59870cdbbd25d7f4d403f6
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
import random
import cv2
from matplotlib import pyplot as plt
import matplotlib.patches as patches
import numpy as np
import albumentations as A
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White
from ftplib import B_CRLF
#import cv2
#import numpy as np
#import albumentations as A
from sklearn.metrics import plot_confusion_matrix
#from utils import plot_examples, visualize
#from pathlib import Path
from PIL import Image
#import matplotlib.pyplot as plt 
import os.path

In [ ]:
path = "/content/drive/My Drive/last_dataset/images/"
path_yolo="/content/drive/My Drive/last_dataset/labels/"

pathImages="/content/drive/MyDrive/fina_fina/images/"
pathTxt="/content/drive/MyDrive/fina_fina/labels/"

train_path_img=pathImages+'train/'
test_path_img=pathImages+'test/'
val_path_img=pathImages+'val/'

train_path_txt=pathTxt+'train/'
test_path_txt=pathTxt+'test/'
val_path_txt=pathTxt+'val/'

In [ ]:
def visualize(image, bboxes, category_ids, category_id_to_name):
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)

def plot_examples(images, bboxes=None):
    fig = plt.figure(figsize=(250,250))
    columns = 4
    rows = 5
    #print(len(images))
    for i in range(1, len(images)):
        if bboxes is not None:
            #print("bboxes[i] : " + str(bboxes[i]))
            img = visualize_bbox(images[i], bboxes[i], class_name="AUnripe")
        else:
            img = images[i]
        fig.add_subplot(rows, columns, i)
        plt.imshow(img)
        plt.axis("off")
    plt.show()


# From https://albumentations.ai/docs/examples/example_bboxes/
def visualize_bbox(img, bbox, class_name, color=BOX_COLOR, thickness=12):
    """Visualizes a single bounding box on the image"""
    x_center, y_center, w, h = bbox
    #print("xmin : " + str(x_min))
    #print("y_min : " + str(y_min))
    #print("w : " + str(w))
    #print("h : " + str(h))
    #x_min, x_max, y_min, y_max = int(x_min - w), int(x_min + w), int(y_min - h), int(y_min + h)
    #print("x_min : " + str(x_min) + " x_max : " + str(x_max))
    #print("y_min : " + str(y_min) + " y_max : " + str(y_max))
    #cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
    dh, dw, _ = img.shape
    #print("dh : {0}; dw : {1}".format(dh,dw))
    #print("x_center : {0}; y_center : {1}; w : {2}; h : {3}".format(x_center, y_center, w, h))
    l = int((x_center - w/2) * dw)
    r = int((x_center + w/2) * dw)
    t = int((y_center - h/2) * dh)
    b = int((y_center + h/2) * dh)
    
    if l < 0: 
        l = 0
    if r > dw - 1:
        r = dw - 1
    if t < 0:
        t = 0
    if b > dh - 1:
        b = dh - 1
    #print("l : {0}, r : {1}, t : {2}, b : {3}".format(l,r,t,b))
    cv2.rectangle(img, (l,t),(r,b),color=color, thickness=thickness)

    """((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.015, 3)
    textSize = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1) 
    #print(textSize)
    #print("text width : " + str(text_width) + " text height : " + str(text_height))
    #print("text width - y_min : " + str(t - 0.3* text_width))
    cv2.rectangle(img, (l, t - int( 0.3* text_height)), (l + text_width, t), BOX_COLOR, 1)
    cv2.rectangle(img, (l, t), (r , b), BOX_COLOR, 1)
    cv2.putText(
        img,
        text=class_name,
        org=(l,t-int(0.3 * text_height) ),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35, 
        color=TEXT_COLOR, 
        lineType=cv2.LINE_AA,
    )"""
    return img


In [ ]:
'''
def check_bbox(bbox):
    """Check if bbox boundaries are in range 0, 1 and minimums are lesser then maximums"""
   #my added block 
    bbox=list(bbox)
    for i in range(4):
      if (bbox[i]<0) :
        bbox[i]=0
      elif (bbox[i]>1):
        bbox[i]=1
    bbox=tuple(bbox)
'''

'\ndef check_bbox(bbox):\n    """Check if bbox boundaries are in range 0, 1 and minimums are lesser then maximums"""\n   #my added block \n    bbox=list(bbox)\n    for i in range(4):\n      if (bbox[i]<0) :\n        bbox[i]=0\n      elif (bbox[i]>1):\n        bbox[i]=1\n    bbox=tuple(bbox)\n'

In [ ]:
def multipleBbox(bboxes,I,img):
  bbox=[];x_min = []; y_min = []; w = []; h = []
  arr=[]
  for i in range (len(bboxes)):
    bbox.append(bboxes[i][1:])
    arr.append(bboxes[i][0])

    '''
  for i in range(len(bbox)):
    x_min.append(bbox[i][1])
    y_min.append(bbox[i][2])
    w.append(bbox[i][3])
    h.append(bbox[i][4])
    '''

    """
  dh, dw, _ = I.shape 
  for i in range(4):
    l = int((x_min[i] - w[i]/2) * dw)
    r = int((x_min[i] + w[i]/2) * dw)
    t = int((y_min[i] - h[i]/2) * dh)
    b = int((y_min[i] + h[i]/2) * dh)
    img = cv2.rectangle(I, (l,t),(r,b),color=(255, 0, 0), thickness=10)
    plt.imshow(img)
    plt.show()
    """
  print(bbox)
  #-----comment2----
  a = augment(I,bbox,img,arr)

In [ ]:
def augment(image, box, imgName,cat):
  int count=0
    transform = A.Compose(
        [
            A.Resize(width=2050, height=2180),
            #A.RandomCrop(width=1280, height=720),
            A.RandomSizedBBoxSafeCrop(height = 1080, width =1080),
            A.Rotate(limit=20),
            A.HorizontalFlip(),
            A.VerticalFlip(),
            A.RandomBrightness(limit = 0.3),
            #A.RGBShift(r_shift_limit=25, g_shift_limit=25, b_shift_limit=25),
            A.OneOf([
                A.Blur(blur_limit=3, p=0.5),
                A.RandomBrightnessContrast()
            ], p=1.0),
        ], bbox_params=A.BboxParams(format="yolo", min_area=2180,
                                min_visibility=0.5, label_fields=[])
    )
    images_list = [image]
    saved_bboxes = [box]
    for i in range(10):

        #check_bbox(box)
        augmentations = transform(image=image, bboxes=box)
        #print(augmentations['bboxes'] )
        #augmentations['bboxes'] = check_bbox(augmentations['bboxes'])
        augmented_img = augmentations["image"]
        if len(augmentations["bboxes"]) == 0:
            continue

        images_list.append(augmented_img)
        saved_bboxes.append(augmentations["bboxes"][0])
       
        fileName = "Aug_"+imgName+"("+str(i+1)+")"+".jpg"


        txtFile = "Aug_"+imgName+"("+str(i+1)+")"+".txt"
        filePath = path+fileName
        txtPath = path_yolo+txtFile
        t = []
        if(len(cat)==1):
          t.append(int(cat[0]))
        
          for j in range(4):
            t.append(augmentations["bboxes"][0][j])
          
        else :
          for i in range(len(cat)):
            t.append(int(cat[i]))
            for j in range(4):
              t.append(augmentations["bboxes"][0][j])


        cv2.imwrite(filePath, cv2.cvtColor(augmented_img, cv2.COLOR_BGR2RGB))
        f = open(txtPath, "w")
        f.write(" ".join(map(str,t)))
        f.close()
        
        #print(type(t))
        #print(t)
        print("Done saving augmentations of the image")
    #plot_examples(images =images_list, bboxes=saved_bboxes)
    #return images_list,saved_bboxes

def openFile(path, imageName):
    #print("In open File")
    bbox = []
    img = "IMG_"+str(imageName) #------------------------------------Image Name--------------------------------------
    i = img +".jpg"
    print("i : ", i)
    filePath = path+i
    #print(filePath)
    I = os.path.exists(filePath)
    #print(I)
    if I:
        I = cv2.imread(filePath)
        I = cv2.cvtColor(I, cv2.COLOR_BGR2RGB)
        #images.append(I)
        #plt.imshow(I)
        #plt.show()
        #b = path+"ARot YOLO txt/" #---------------------------------------------File name ------------------------------------------------------
        b = path_yolo+img+".txt"
        print(b)
        bboxes = np.loadtxt(b, delimiter=' ')
        bboxes = bboxes.tolist()
        #print(bboxes)
        print(len(bboxes))
        '''
        if len(bboxes) > 1 :
          multipleBbox(bboxes,I,img)
        else:
          #b =[bboxes]
          bbox.append(bboxes[0][1:])
          print(bbox)
         
        '''
        cat=[]
        cat.append(bboxes[0])

        bbox.append(bboxes[1:])
        #plot_examples(I,bbox)
          #x_min, y_min, w, h = bbox[1:]
        a = augment(I,bbox,img,cat)
        #print(bbox)
        
         #-----------------comment1------------
        '''
        x_min, y_min, w, h = bbox[0]
        
        
        dh, dw, _ = I.shape 
        l = int((x_min - w/2) * dw)
        r = int((x_min + w/2) * dw)
        t = int((y_min - h/2) * dh)
        b = int((y_min + h/2) * dh)
        img = cv2.rectangle(I, (l,t),(r,b),color=(255, 0, 0), thickness=10)
        plt.imshow(img)
        plt.show()
        
        
                #pixel resolution of images 3042x4032
        #print("a :" ,"".join(str(a)))
        #print("".join(str(a[1])))
        plot_examples(images =a[0], bboxes=a[1])
        visualize(image = a[0], bboxes = a[1], category_ids=[0],category_id_to_name="Aunripe")
        '''




for i in range(990,1285):   #-----------------------------------------------Image Name ------------------------------------------
    #print("I : " +str(i))
    #print("i :" +  str(i))
    openFile(path,i)
#os.listdir(path)

Streaming output truncated to the last 5000 lines.
Done saving augmentations of the image
[0.378348 0.580357 0.634921 0.426587]
Done saving augmentations of the image
[0.378348 0.580357 0.634921 0.426587]
Done saving augmentations of the image
[0.378348 0.580357 0.634921 0.426587]
Done saving augmentations of the image
[0.378348 0.580357 0.634921 0.426587]
Done saving augmentations of the image
i :  IMG_1068.jpg
/content/drive/My Drive/last_dataset/labels/IMG_1068.txt
5
[0.555827 0.421627 0.474702 0.724206]
Done saving augmentations of the image
[0.555827 0.421627 0.474702 0.724206]
Done saving augmentations of the image
[0.555827 0.421627 0.474702 0.724206]
Done saving augmentations of the image
[0.555827 0.421627 0.474702 0.724206]
Done saving augmentations of the image
[0.555827 0.421627 0.474702 0.724206]
Done saving augmentations of the image
[0.555827 0.421627 0.474702 0.724206]
Done saving augmentations of the image
[0.555827 0.421627 0.474702 0.724206]
Done saving augmentations

# Not augmented images

1. 13_2_22(Day 7) picture 8,10 
2. 14_2_22 (Day 8) picture 11
3. 15_2_22 (Day 9) picture 2,8 
4. 17_2_22 (Day 10) picture 14
5. 18_2_22 (Day 11) 1645165746630 , 1645165746656